## Final Project Submission

Please fill out:
* Student name: Samuel Marder
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: Matt Bombard
* Blog post URL:


# Overview

# Business Understanding

Century 21 is looking to expand its housing footprint. This would be made a lot easier having a regression model to work from

# Data Understanding

# Data Analysis

In [1]:
import pandas as pd
df = pd.read_csv("data/kc_house_data.csv", index_col="id")
df

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,
7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,Average,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,Average,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,Average,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,Very Good,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,Average,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263000018,5/21/2014,360000.0,3,2.50,1530,1131,3.0,NO,NONE,Average,8 Good,1530,0.0,2009,0.0,98103,47.6993,-122.346,1530,1509
6600060120,2/23/2015,400000.0,4,2.50,2310,5813,2.0,NO,NONE,Average,8 Good,2310,0.0,2014,0.0,98146,47.5107,-122.362,1830,7200
1523300141,6/23/2014,402101.0,2,0.75,1020,1350,2.0,NO,NONE,Average,7 Average,1020,0.0,2009,0.0,98144,47.5944,-122.299,1020,2007


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21597 entries, 7129300520 to 1523300157
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           21597 non-null  object 
 1   price          21597 non-null  float64
 2   bedrooms       21597 non-null  int64  
 3   bathrooms      21597 non-null  float64
 4   sqft_living    21597 non-null  int64  
 5   sqft_lot       21597 non-null  int64  
 6   floors         21597 non-null  float64
 7   waterfront     19221 non-null  object 
 8   view           21534 non-null  object 
 9   condition      21597 non-null  object 
 10  grade          21597 non-null  object 
 11  sqft_above     21597 non-null  int64  
 12  sqft_basement  21597 non-null  object 
 13  yr_built       21597 non-null  int64  
 14  yr_renovated   17755 non-null  float64
 15  zipcode        21597 non-null  int64  
 16  lat            21597 non-null  float64
 17  long           21597 non-null  float

Setting up our X and y with one hot encoder and getting dummies

In [3]:
from sklearn.model_selection import train_test_split
X = df.drop(["price"], axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [4]:
relevant_cols = [
    "bedrooms",		# Number of bedrooms
    "bathrooms",	# Number of bathrooms
    "sqft_living",	# Square foot of living space
    "sqft_lot",		# Square foot of the lot
    "floors",		# Number of floors
    "waterfront",	# If there is a waterfront view
    "condition",	# How good the overall condition of the house is
    "grade",		# Overall grade of the house
    "yr_built"]		# Year the house was built in

X_train = X_train[relevant_cols]
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16197 entries, 3333002440 to 5668500045
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   bedrooms     16197 non-null  int64  
 1   bathrooms    16197 non-null  float64
 2   sqft_living  16197 non-null  int64  
 3   sqft_lot     16197 non-null  int64  
 4   floors       16197 non-null  float64
 5   waterfront   14420 non-null  object 
 6   condition    16197 non-null  object 
 7   grade        16197 non-null  object 
 8   yr_built     16197 non-null  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 1.2+ MB


Test for nulls first to see if we can impute the missing values

In [5]:
print(X_train.isna().sum())

bedrooms          0
bathrooms         0
sqft_living       0
sqft_lot          0
floors            0
waterfront     1777
condition         0
grade             0
yr_built          0
dtype: int64


In [6]:
X_train.waterfront.fillna(value="NO", inplace=True)
print(X_train.isna().sum())

bedrooms       0
bathrooms      0
sqft_living    0
sqft_lot       0
floors         0
waterfront     0
condition      0
grade          0
yr_built       0
dtype: int64


In [7]:
print(X_train.condition.value_counts())
print()
print(X_train.waterfront.value_counts())
print()
print(X_train.grade.value_counts())

Average      10499
Good          4259
Very Good     1283
Fair           132
Poor            24
Name: condition, dtype: int64

NO     16081
YES      116
Name: waterfront, dtype: int64

7 Average        6713
8 Good           4551
9 Better         1982
6 Low Average    1537
10 Very Good      824
11 Excellent      305
5 Fair            181
12 Luxury          68
4 Low              23
13 Mansion         12
3 Poor              1
Name: grade, dtype: int64


In [8]:
def waterfront_to_num(wf):
    if wf == "YES":
        return 1
    return 0

def condition_to_num(condition):
    if condition == "Average":
        return 3
    elif condition == "Good":
        return 4
    elif condition == "Very Good":
        return 5
    elif condition == "Fair":
        return 2
    return 1

In [9]:
X_train['waterfront'] = X_train.waterfront.apply(waterfront_to_num)

X_train['condition'] = X_train.condition.apply(condition_to_num)

X_train['grade'].replace(to_replace=r'\D', value=r'', regex=True, inplace=True)
X_train['grade'] = X_train["grade"].astype(int)

print(X_train.condition.value_counts())
print()
print(X_train.waterfront.value_counts())
print()
print(X_train.grade.value_counts())

3    10499
4     4259
5     1283
2      132
1       24
Name: condition, dtype: int64

0    16081
1      116
Name: waterfront, dtype: int64

7     6713
8     4551
9     1982
6     1537
10     824
11     305
5      181
12      68
4       23
13      12
3        1
Name: grade, dtype: int64


In [10]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [11]:
from sklearn.model_selection import cross_val_score

cross_val_score(model, X_train, y_train, cv=3)

array([0.64027696, 0.65188193, 0.64246979])

In [12]:
X_test = X_test[relevant_cols]

X_test.waterfront.fillna(value="NO", inplace=True)
X_test['waterfront'] = X_test.waterfront.apply(waterfront_to_num)

X_test['condition'] = X_test.condition.apply(condition_to_num)

X_test['grade'].replace(to_replace=r'\D', value=r'', regex=True, inplace=True)
X_test['grade'] = X_test["grade"].astype(int)

In [13]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5400 entries, 7305300695 to 1250200595
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   bedrooms     5400 non-null   int64  
 1   bathrooms    5400 non-null   float64
 2   sqft_living  5400 non-null   int64  
 3   sqft_lot     5400 non-null   int64  
 4   floors       5400 non-null   float64
 5   waterfront   5400 non-null   int64  
 6   condition    5400 non-null   int64  
 7   grade        5400 non-null   int32  
 8   yr_built     5400 non-null   int64  
dtypes: float64(2), int32(1), int64(6)
memory usage: 400.8 KB


In [14]:
model.score(X_test, y_test)

0.6399730520347621

In [15]:
model.coef_

array([-3.96519695e+04,  5.10323736e+04,  1.69034073e+02, -2.22234439e-01,
        2.35933771e+04,  6.58684018e+05,  2.05547798e+04,  1.30426952e+05,
       -3.89236635e+03])